# 0. Carregando os dados e bibliotecas

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
data = pd.read_csv('datasets/kc_house_data.csv')

# 1. Respondendo às perguntas de negócio

## 1.1 Qual o número de imóveis por ano de construção?

In [ ]:
data.groupby('yr_built')[['id']].count()

#### Outra forma de fazer

In [ ]:
data[['id', 'yr_built']].groupby('yr_built').count()

#essa sintaxe pode ser usada para os exercícios da seção 1

## 1.2 Qual o menor número de quartos por ano de construção dos imóveis?


In [ ]:
data.groupby('yr_built')[['bedrooms']].min()

## 1.3 Qual o preço de compra mais alto por cada número de quartos?

In [ ]:
data.groupby('bedrooms')[['price']].max()

## 1.4 Qual a soma de todos os preços de compra por cada número de quartos?

In [ ]:
data.groupby('bedrooms')[['price']].sum()

## 1.5 Qual a soma de todos os preços de compra pelo número de quartos e banheiros?

In [ ]:
data.groupby(['bedrooms', 'bathrooms'])[['price']].sum()

## 1.6 Qual o tamanho médio das salas dos imóveis por ano de construção?

In [ ]:
data.groupby('yr_built')[['sqft_living']].mean()

## 1.7 . Qual o tamanho mediano das salas dos imóveis por ano de construção?

## 1.8 Qual o desvio-padrão do tamanho das salas dos imóveis por ano de construção?

## 1.9 Como é o crescimento médio preços de compra dos imóveis, por dia e semana do ano?

## 1.10 Eu gostaria de olhar no mapa e conseguir identificar as casas com o maior preço.